In [ ]:
# === Setup ===
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import sys
import time

# Add repo root to path
sys.path.insert(0, str(Path.cwd().parent.parent.parent))

from modules._import_helper import safe_import_from

# Import utilities
set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')
(HarmonicOscillatorConfig, HarmonicOscillatorPINN,
 solve_harmonic_oscillator_scipy, analytical_harmonic_oscillator,
 compute_energy) = safe_import_from(
    '07_physics_informed_ml.src.ode_pinn',
    'HarmonicOscillatorConfig', 'HarmonicOscillatorPINN',
    'solve_harmonic_oscillator_scipy', 'analytical_harmonic_oscillator',
    'compute_energy'
)

# Matplotlib config
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Reports directory
reports_dir = Path.cwd().parent / 'reports'
reports_dir.mkdir(exist_ok=True)

# Reproducibility
SEED = 42
set_seed(SEED)
torch.manual_seed(SEED)

print(f"PyTorch version: {torch.__version__}")
print(f"Reports will be saved to: {reports_dir}")
print("✓ Setup complete")

---
## 1. The Physics Problem

### Harmonic Oscillator ODE

$$\frac{d^2x}{dt^2} + \omega^2 x = 0$$

**Physical interpretation**: Mass on spring, pendulum (small angle), LC circuit.

**First-order system** (for numerical methods):
$$\begin{cases} \frac{dx}{dt} = v \\ \frac{dv}{dt} = -\omega^2 x \end{cases}$$

**Initial conditions**: $x(0) = x_0$, $\dot{x}(0) = v_0$

### Analytical Solution

$$x(t) = x_0 \cos(\omega t) + \frac{v_0}{\omega} \sin(\omega t)$$

$$v(t) = -x_0 \omega \sin(\omega t) + v_0 \cos(\omega t)$$

### Energy Conservation

$$E = \frac{1}{2}v^2 + \frac{1}{2}\omega^2 x^2 = \text{constant}$$

In [ ]:
# === Visualize ground truth ===
omega = 1.0
x0, v0 = 1.0, 0.0
t_eval = np.linspace(0, 10, 500)

x_exact, v_exact = analytical_harmonic_oscillator(omega, x0, v0, t_eval)
E_exact = compute_energy(x_exact, v_exact, omega)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].plot(t_eval, x_exact, 'k-', lw=2, label='x(t)')
axes[0].plot(t_eval, v_exact, 'b--', lw=2, label='v(t)')
axes[0].set_xlabel('Time t')
axes[0].set_ylabel('State')
axes[0].set_title('Trajectory')
axes[0].legend()

axes[1].plot(x_exact, v_exact, 'k-', lw=2)
axes[1].plot(x0, v0, 'go', ms=10, label=f'IC: ({x0}, {v0})')
axes[1].set_xlabel('Position x')
axes[1].set_ylabel('Velocity v')
axes[1].set_title('Phase Space')
axes[1].axis('equal')
axes[1].legend()

axes[2].plot(t_eval, E_exact, 'k-', lw=2)
axes[2].set_xlabel('Time t')
axes[2].set_ylabel('Energy E')
axes[2].set_title(f'Energy Conservation (E = {E_exact[0]:.4f})')
axes[2].set_ylim([E_exact[0] - 0.1, E_exact[0] + 0.1])

plt.tight_layout()
plt.savefig(reports_dir / '01_harmonic_ground_truth.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Period T = 2π/ω = {2*np.pi/omega:.4f}")

---
## 2. PINN Architecture

### Network
$$\hat{x}(t) = \text{MLP}(t; \theta)$$

Input: $t$ (scalar time)  
Output: $\hat{x}(t)$ (predicted position)

### Physics Loss (ODE Residual)

We use PyTorch autograd to compute derivatives:

$$\hat{v} = \frac{\partial \hat{x}}{\partial t}, \quad \hat{a} = \frac{\partial^2 \hat{x}}{\partial t^2}$$

The ODE residual:
$$r(t) = \hat{a} + \omega^2 \hat{x} = \frac{\partial^2 \hat{x}}{\partial t^2} + \omega^2 \hat{x}$$

$$\mathcal{L}_{\text{physics}} = \frac{1}{N_c} \sum_{i=1}^{N_c} r(t_i)^2$$

### Initial Condition Loss

$$\mathcal{L}_{\text{IC}} = (\hat{x}(0) - x_0)^2 + (\hat{v}(0) - v_0)^2$$

### Total Loss

$$\mathcal{L}_{\text{total}} = \lambda_{\text{physics}} \mathcal{L}_{\text{physics}} + \lambda_{\text{IC}} \mathcal{L}_{\text{IC}}$$

In [ ]:
# === Train baseline PINN ===
set_seed(SEED)

config = HarmonicOscillatorConfig(
    omega=1.0,
    x0=1.0,
    v0=0.0,
    t_max=10.0,
    n_collocation=200,
    hidden_dims=[32, 32, 32],
    epochs=5000,
    lr=1e-3,
    lambda_physics=1.0,
    lambda_ic=10.0,
)

pinn = HarmonicOscillatorPINN(config)

print(f"Network architecture: 1 → {config.hidden_dims} → 1")
print(f"Collocation points: {config.n_collocation}")
print(f"Loss weights: λ_physics={config.lambda_physics}, λ_IC={config.lambda_ic}")
print("\nTraining...")

start_time = time.time()
history = pinn.train(verbose=1000)
train_time = time.time() - start_time

print(f"\n✓ Training completed in {train_time:.2f}s")
print(f"Final loss: {history['loss'][-1]:.6f}")

In [ ]:
# === Compare to baseline: scipy solve_ivp ===
t_test = np.linspace(0, config.t_max, 200)

# PINN prediction
x_pinn, v_pinn = pinn.predict_with_velocity(t_test)

# scipy baseline
start_time = time.time()
x_scipy, v_scipy = solve_harmonic_oscillator_scipy(config.omega, config.x0, config.v0, t_test)
scipy_time = time.time() - start_time

# Analytical
x_analytical, v_analytical = analytical_harmonic_oscillator(
    config.omega, config.x0, config.v0, t_test
)

# Compute errors
def compute_errors(x_pred, x_true):
    rmse = np.sqrt(np.mean((x_pred - x_true)**2))
    max_err = np.max(np.abs(x_pred - x_true))
    rel_err = rmse / np.std(x_true) * 100
    return rmse, max_err, rel_err

pinn_rmse, pinn_max, pinn_rel = compute_errors(x_pinn, x_analytical)
scipy_rmse, scipy_max, scipy_rel = compute_errors(x_scipy, x_analytical)

print("="*60)
print("ACCURACY COMPARISON")
print("="*60)
print(f"{'Metric':<20} {'PINN':>15} {'scipy (RK45)':>15}")
print("-"*60)
print(f"{'RMSE':<20} {pinn_rmse:>15.6f} {scipy_rmse:>15.2e}")
print(f"{'Max Error':<20} {pinn_max:>15.6f} {scipy_max:>15.2e}")
print(f"{'Relative Error (%)':<20} {pinn_rel:>15.2f} {scipy_rel:>15.2e}")
print("-"*60)
print(f"{'Compute Time (s)':<20} {train_time:>15.2f} {scipy_time:>15.4f}")
print("="*60)

In [ ]:
# === Visualization ===
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training convergence
ax = axes[0, 0]
ax.semilogy(history['loss'], 'k-', lw=2, label='Total')
ax.semilogy(history['loss_physics'], 'b--', lw=1.5, alpha=0.8, label='Physics')
ax.semilogy(history['loss_ic'], 'r:', lw=1.5, alpha=0.8, label='IC')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Convergence')
ax.legend()

# Solution comparison
ax = axes[0, 1]
ax.plot(t_test, x_analytical, 'k-', lw=2.5, label='Analytical', alpha=0.7)
ax.plot(t_test, x_scipy, 'b--', lw=2, label='scipy (RK45)')
ax.plot(t_test, x_pinn, 'r:', lw=2.5, label='PINN')
ax.set_xlabel('Time t')
ax.set_ylabel('Position x(t)')
ax.set_title('Solution Comparison')
ax.legend()

# Pointwise error
ax = axes[1, 0]
error = np.abs(x_pinn - x_analytical)
ax.semilogy(t_test, error, 'r-', lw=2)
ax.axhline(pinn_rmse, color='b', ls='--', lw=1.5, label=f'RMSE = {pinn_rmse:.6f}')
ax.set_xlabel('Time t')
ax.set_ylabel('|Error|')
ax.set_title('Pointwise Absolute Error')
ax.legend()

# Energy conservation
ax = axes[1, 1]
E_analytical = compute_energy(x_analytical, v_analytical, config.omega)
E_pinn = compute_energy(x_pinn, v_pinn, config.omega)
ax.plot(t_test, E_analytical, 'k-', lw=2, label='Analytical')
ax.plot(t_test, E_pinn, 'r:', lw=2.5, label='PINN')
ax.axhline(E_analytical[0], color='gray', ls='--', alpha=0.5)
ax.set_xlabel('Time t')
ax.set_ylabel('Energy E')
ax.set_title(f'Energy Conservation (violation: {np.std(E_pinn)/np.mean(E_pinn)*100:.2f}%)')
ax.legend()

plt.tight_layout()
plt.savefig(reports_dir / '01_pinn_vs_baseline.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 3. Experiment: Effect of Collocation Points

**Question**: How does the number of collocation points affect accuracy?

In [ ]:
# === Experiment 1: Number of collocation points ===
n_collocation_values = [20, 50, 100, 200, 500]
results_ncol = []

print("Experiment: Effect of N_collocation")
print("="*50)

for n_col in n_collocation_values:
    set_seed(SEED)
    
    config_test = HarmonicOscillatorConfig(
        omega=1.0, x0=1.0, v0=0.0, t_max=10.0,
        n_collocation=n_col,
        hidden_dims=[32, 32, 32],
        epochs=3000,
        lr=1e-3,
        lambda_physics=1.0,
        lambda_ic=10.0,
    )
    
    pinn_test = HarmonicOscillatorPINN(config_test)
    history_test = pinn_test.train(verbose=0)
    
    x_pred, _ = pinn_test.predict_with_velocity(t_test)
    rmse, max_err, rel_err = compute_errors(x_pred, x_analytical)
    
    results_ncol.append({
        'n_col': n_col,
        'rmse': rmse,
        'max_err': max_err,
        'rel_err': rel_err,
        'final_loss': history_test['loss'][-1],
    })
    
    print(f"N_col={n_col:4d} | RMSE={rmse:.6f} | Max={max_err:.6f} | Rel={rel_err:.2f}%")

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
n_cols = [r['n_col'] for r in results_ncol]
rmses = [r['rmse'] for r in results_ncol]

ax.semilogy(n_cols, rmses, 'ko-', lw=2, ms=8)
ax.set_xlabel('Number of Collocation Points')
ax.set_ylabel('RMSE')
ax.set_title('Effect of Collocation Points on Accuracy')
plt.tight_layout()
plt.savefig(reports_dir / '01_exp_collocation.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Experiment: Effect of Loss Weighting

**Question**: How does $\lambda_{IC}$ affect IC satisfaction vs physics residual?

In [ ]:
# === Experiment 2: Loss weighting λ_IC ===
lambda_ic_values = [0.1, 1.0, 10.0, 100.0, 1000.0]
results_lambda = []

print("Experiment: Effect of λ_IC")
print("="*70)

for lam_ic in lambda_ic_values:
    set_seed(SEED)
    
    config_test = HarmonicOscillatorConfig(
        omega=1.0, x0=1.0, v0=0.0, t_max=10.0,
        n_collocation=200,
        hidden_dims=[32, 32, 32],
        epochs=3000,
        lr=1e-3,
        lambda_physics=1.0,
        lambda_ic=lam_ic,
    )
    
    pinn_test = HarmonicOscillatorPINN(config_test)
    history_test = pinn_test.train(verbose=0)
    
    x_pred, v_pred = pinn_test.predict_with_velocity(t_test)
    rmse, max_err, rel_err = compute_errors(x_pred, x_analytical)
    
    # IC error
    ic_x_err = np.abs(x_pred[0] - config.x0)
    ic_v_err = np.abs(v_pred[0] - config.v0)
    
    results_lambda.append({
        'lambda_ic': lam_ic,
        'rmse': rmse,
        'ic_x_err': ic_x_err,
        'ic_v_err': ic_v_err,
        'loss_physics': history_test['loss_physics'][-1],
        'loss_ic': history_test['loss_ic'][-1],
    })
    
    print(f"λ_IC={lam_ic:6.1f} | RMSE={rmse:.6f} | IC_x_err={ic_x_err:.4f} | IC_v_err={ic_v_err:.4f}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

lambdas = [r['lambda_ic'] for r in results_lambda]
rmses = [r['rmse'] for r in results_lambda]
ic_errs = [r['ic_x_err'] + r['ic_v_err'] for r in results_lambda]

axes[0].semilogx(lambdas, rmses, 'ko-', lw=2, ms=8)
axes[0].set_xlabel('λ_IC')
axes[0].set_ylabel('RMSE')
axes[0].set_title('Overall Accuracy vs λ_IC')

axes[1].loglog(lambdas, ic_errs, 'ro-', lw=2, ms=8)
axes[1].set_xlabel('λ_IC')
axes[1].set_ylabel('IC Error (|Δx| + |Δv|)')
axes[1].set_title('Initial Condition Violation vs λ_IC')

plt.tight_layout()
plt.savefig(reports_dir / '01_exp_lambda_ic.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n⚠️ Too small λ_IC → IC violated. Too large → training unstable.")

---
## 5. Experiment: High Frequency Challenge

**Question**: Can PINN handle higher frequencies (more oscillations)?

In [ ]:
# === Experiment 3: Frequency sensitivity ===
omega_values = [0.5, 1.0, 2.0, 4.0, 8.0]
results_freq = []

print("Experiment: Effect of Frequency ω")
print("="*60)

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for idx, omega_test in enumerate(omega_values):
    set_seed(SEED)
    
    config_test = HarmonicOscillatorConfig(
        omega=omega_test,
        x0=1.0, v0=0.0, t_max=10.0,
        n_collocation=200,
        hidden_dims=[32, 32, 32],
        epochs=3000,
        lr=1e-3,
        lambda_physics=1.0,
        lambda_ic=10.0,
    )
    
    pinn_test = HarmonicOscillatorPINN(config_test)
    history_test = pinn_test.train(verbose=0)
    
    x_pred, _ = pinn_test.predict_with_velocity(t_test)
    x_true, _ = analytical_harmonic_oscillator(omega_test, 1.0, 0.0, t_test)
    rmse, max_err, rel_err = compute_errors(x_pred, x_true)
    
    n_oscillations = omega_test * 10 / (2 * np.pi)
    
    results_freq.append({
        'omega': omega_test,
        'n_osc': n_oscillations,
        'rmse': rmse,
        'rel_err': rel_err,
    })
    
    # Plot
    ax = axes[idx]
    ax.plot(t_test, x_true, 'k-', lw=2, label='Analytical')
    ax.plot(t_test, x_pred, 'r:', lw=2, label='PINN')
    ax.set_title(f'ω={omega_test} ({n_oscillations:.1f} osc.)\nRMSE={rmse:.4f}')
    ax.set_xlabel('Time t')
    if idx % 3 == 0:
        ax.set_ylabel('x(t)')
    ax.legend(fontsize=8)
    
    print(f"ω={omega_test:4.1f} | {n_oscillations:5.1f} osc | RMSE={rmse:.6f} | Rel={rel_err:.2f}%")

# Summary in last subplot
ax = axes[-1]
omegas = [r['omega'] for r in results_freq]
rmses = [r['rmse'] for r in results_freq]
ax.semilogy(omegas, rmses, 'ko-', lw=2, ms=8)
ax.set_xlabel('Frequency ω')
ax.set_ylabel('RMSE')
ax.set_title('Error vs Frequency')

plt.tight_layout()
plt.savefig(reports_dir / '01_exp_frequency.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n⚠️ Higher frequencies → more oscillations → harder to learn!")
print("   Solution: more collocation points, larger network, or Fourier features.")

---
## 6. Summary Results Table

In [ ]:
# === Save summary to reports ===
summary = f"""
# PINN ODE: Harmonic Oscillator - Results Summary

**Date**: {time.strftime('%Y-%m-%d %H:%M')}
**Seed**: {SEED}

## Baseline Comparison

| Method | RMSE | Max Error | Compute Time |
|--------|------|-----------|-------------|
| PINN | {pinn_rmse:.6f} | {pinn_max:.6f} | {train_time:.2f}s |
| scipy RK45 | {scipy_rmse:.2e} | {scipy_max:.2e} | {scipy_time:.4f}s |

## Collocation Points Sensitivity

| N_col | RMSE |
|-------|------|
""" + "\n".join([f"| {r['n_col']} | {r['rmse']:.6f} |" for r in results_ncol]) + """

## Loss Weight Sensitivity (λ_IC)

| λ_IC | RMSE | IC Error |
|------|------|----------|
""" + "\n".join([f"| {r['lambda_ic']:.1f} | {r['rmse']:.6f} | {r['ic_x_err']+r['ic_v_err']:.4f} |" for r in results_lambda]) + """

## Frequency Sensitivity

| ω | # Oscillations | RMSE |
|---|----------------|------|
""" + "\n".join([f"| {r['omega']:.1f} | {r['n_osc']:.1f} | {r['rmse']:.6f} |" for r in results_freq]) + """

## Key Findings

1. **PINN achieves ~1% relative error** with proper hyperparameters
2. **N_collocation**: Diminishing returns past ~100-200 points
3. **λ_IC**: Sweet spot around 10-100; too low violates IC, too high hinders convergence
4. **High frequencies are hard**: Error increases with ω due to spectral bias
"""

with open(reports_dir / '01_pinn_ode_summary.md', 'w') as f:
    f.write(summary)

print(summary)
print(f"\n✓ Summary saved to {reports_dir / '01_pinn_ode_summary.md'}")

---
## 7. Failure Modes & Debugging Checklist

### Common Failures

| Symptom | Likely Cause | Fix |
|---------|--------------|-----|
| IC violated | λ_IC too small | Increase λ_IC (try 10-100) |
| Solution drifts over time | Not enough collocation points | Increase N_col or use adaptive sampling |
| High-frequency modes missed | Spectral bias | Use Fourier features or larger network |
| Training stuck | Learning rate too high/low | Use scheduler or tune LR |
| NaN losses | Gradient explosion | Reduce LR, add gradient clipping |

### Debugging Steps

1. **Check IC satisfaction first**: Plot x(0) and v(0) vs targets
2. **Monitor loss components**: Physics vs IC loss - both should decrease
3. **Visualize residual field**: Plot |r(t)| - should be small everywhere
4. **Check gradient magnitudes**: `torch.nn.utils.clip_grad_norm_`
5. **Try input normalization**: Scale t to [0, 1]

---
## 8. Mini Exercises

**Exercise 1**: Train a PINN with `v0 = 1.0` (non-zero initial velocity). How does the solution change?

**Exercise 2**: Add damping: $\ddot{x} + 2\gamma\dot{x} + \omega^2 x = 0$. Modify the physics residual.

**Exercise 3**: Try `t_max = 50.0`. What happens? How can you fix it?

**Exercise 4**: Implement input scaling: $\tilde{t} = t / t_{max}$. Does it improve training?

**Exercise 5**: Plot the residual $|r(t)|$ over the domain. Where is it largest?

In [ ]:
# === Exercise 1: Non-zero initial velocity ===
# YOUR CODE HERE


In [ ]:
# === Exercise 2: Damped oscillator ===
# Hint: Modify physics_residual to include damping term
# YOUR CODE HERE


In [ ]:
# === Exercise 3: Longer time horizon ===
# YOUR CODE HERE


In [ ]:
# === Exercise 4: Input scaling ===
# YOUR CODE HERE


In [ ]:
# === Exercise 5: Residual visualization ===
# YOUR CODE HERE


---
## Solutions (Expand after attempting exercises)

In [ ]:
# === Solution 1: Non-zero initial velocity ===
# Uncomment to see solution:

# set_seed(42)
# config_ex1 = HarmonicOscillatorConfig(
#     omega=1.0, x0=1.0, v0=1.0,  # <-- Changed v0
#     t_max=10.0, n_collocation=200, epochs=3000
# )
# pinn_ex1 = HarmonicOscillatorPINN(config_ex1)
# _ = pinn_ex1.train(verbose=0)
# x_ex1, v_ex1 = pinn_ex1.predict_with_velocity(t_test)
# x_true_ex1, _ = analytical_harmonic_oscillator(1.0, 1.0, 1.0, t_test)
#
# plt.figure(figsize=(8, 4))
# plt.plot(t_test, x_true_ex1, 'k-', lw=2, label='Analytical')
# plt.plot(t_test, x_ex1, 'r:', lw=2, label='PINN')
# plt.xlabel('Time t'); plt.ylabel('x(t)')
# plt.title('Exercise 1: v0 = 1.0')
# plt.legend(); plt.show()

In [ ]:
# === Solution 5: Residual visualization ===
# Uncomment to see solution:

# from modules._import_helper import safe_import_from
# compute_gradient = safe_import_from('07_physics_informed_ml.src.pinn_base', 'compute_gradient')
#
# t_dense = np.linspace(0, config.t_max, 500)
# t_tensor = torch.tensor(t_dense, dtype=torch.float32).view(-1, 1)
# t_tensor.requires_grad = True
#
# x_tensor = pinn.model(t_tensor)
# v_tensor = compute_gradient(x_tensor, t_tensor, order=1)
# a_tensor = compute_gradient(x_tensor, t_tensor, order=2)
#
# residual = (a_tensor + config.omega**2 * x_tensor).detach().numpy().flatten()
#
# plt.figure(figsize=(8, 4))
# plt.semilogy(t_dense, np.abs(residual), 'r-', lw=2)
# plt.xlabel('Time t'); plt.ylabel('|Residual|')
# plt.title('Physics Residual Over Domain')
# plt.show()

---
## Key Takeaways

### ✅ What We Learned

1. **PINNs can solve ODEs without data** by minimizing physics residuals
2. **Loss balancing is critical**: λ_IC controls IC satisfaction vs physics fit
3. **Collocation density matters**: More points → better coverage, but diminishing returns
4. **Spectral bias limits high-frequency accuracy**: NNs struggle with rapid oscillations

### ⚠️ Limitations

1. **Much slower than scipy**: Training takes seconds, scipy takes milliseconds
2. **Hyperparameter sensitive**: λ weights, architecture, LR all matter
3. **Energy not exactly conserved** (unless explicitly enforced)
4. **Long time horizons degrade** without curriculum learning

### 💡 When to Use PINNs for ODEs?

- **Inverse problems**: Discover unknown parameters from sparse data
- **Coupled multi-physics**: When equations are complex to code
- **NOT for**: Simple ODEs where scipy works perfectly!